In [59]:
library(repr)
library(tidyverse)
library(testthat)
library(digest)
library(dplyr)

In [23]:
install.packages("ggmap")

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/3.5’
(as ‘lib’ is unspecified)
also installing the dependencies ‘RgoogleMaps’, ‘png’, ‘rjson’, ‘jpeg’



In [24]:
library(ggmap)

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap") for details.


In [2]:
data <- read_csv("2010-2018data.csv",col_names = TRUE, skip=2)
data <- na.omit(data) 
colnames(data) = c("year","destination","origin","refuge","asylum")

Parsed with column specification:
cols(
  Year = col_double(),
  `Country / territory of asylum/residence` = col_character(),
  Origin = col_character(),
  `Refugees (incl. refugee-like situations)` = col_double(),
  `Asylum-seekers (pending cases)` = col_double()
)
Warning message:
“4198 parsing failures.
  row                                      col expected actual                file
52078 Asylum-seekers (pending cases)           a double      * '2010-2018data.csv'
52081 Refugees (incl. refugee-like situations) a double      * '2010-2018data.csv'
52084 Refugees (incl. refugee-like situations) a double      * '2010-2018data.csv'
52085 Asylum-seekers (pending cases)           a double      * '2010-2018data.csv'
52086 Asylum-seekers (pending cases)           a double      * '2010-2018data.csv'
..... ........................................ ........ ...... ...................
See problems(...) for more details.
”

In [8]:
head(data)

year,destination,origin,refuge,asylum
2010,Afghanistan,Iran (Islamic Rep. of),30,21
2010,Afghanistan,Pakistan,6398,9
2010,Angola,Burundi,14,13
2010,Angola,Central African Rep.,6,1
2010,Angola,Côte d'Ivoire,211,1213
2010,Angola,Dem. Rep. of the Congo,13648,1508


In [9]:
lapply(data,class)

$year
[1] "numeric"

$destination
[1] "character"

$origin
[1] "character"

$refuge
[1] "numeric"

$asylum
[1] "numeric"

In [13]:
data <- data %>%
    mutate(total = refuge + asylum) %>%
    filter(total > 0)

In [14]:
head(data)

year,destination,origin,refuge,asylum,total
2010,Afghanistan,Iran (Islamic Rep. of),30,21,51
2010,Afghanistan,Pakistan,6398,9,6407
2010,Angola,Burundi,14,13,27
2010,Angola,Central African Rep.,6,1,7
2010,Angola,Côte d'Ivoire,211,1213,1424
2010,Angola,Dem. Rep. of the Congo,13648,1508,15156


In [68]:
largest<-arrange(data,desc(total))

In [71]:
head(largest,20)

year,destination,origin,refuge,asylum,total
2018,Turkey,Syrian Arab Rep.,3622366,112,3622478
2017,Turkey,Syrian Arab Rep.,3424237,163,3424400
2016,Turkey,Syrian Arab Rep.,2823987,180,2824167
2015,Turkey,Syrian Arab Rep.,2503549,297,2503846
2010,Pakistan,Afghanistan,1899842,1946,1901788
2011,Pakistan,Afghanistan,1701945,1543,1703488
2012,Pakistan,Afghanistan,1637740,3220,1640960
2013,Pakistan,Afghanistan,1615876,5323,1621199
2015,Pakistan,Afghanistan,1560592,6358,1566950
2014,Turkey,Syrian Arab Rep.,1557899,250,1558149


In [74]:
syriain<-filter(data,destination=="Syrian Arab Rep.")
syriaout<-filter(data,origin=="Syrian Arab Rep.")

In [77]:
head(arrange(syriain,desc(total)))
head(arrange(syriaout,desc(total)))

year,destination,origin,refuge,asylum,total
2010,Syrian Arab Rep.,Iraq,1000000,623,1000623
2011,Syrian Arab Rep.,Iraq,750000,892,750892
2012,Syrian Arab Rep.,Iraq,471418,693,472111
2014,Syrian Arab Rep.,Iraq,146200,1214,147414
2013,Syrian Arab Rep.,Iraq,146200,878,147078
2018,Syrian Arab Rep.,Iraq,15699,18341,34040


year,destination,origin,refuge,asylum,total
2018,Turkey,Syrian Arab Rep.,3622366,112,3622478
2017,Turkey,Syrian Arab Rep.,3424237,163,3424400
2016,Turkey,Syrian Arab Rep.,2823987,180,2824167
2015,Turkey,Syrian Arab Rep.,2503549,297,2503846
2014,Turkey,Syrian Arab Rep.,1557899,250,1558149
2014,Lebanon,Syrian Arab Rep.,1147494,417,1147911


In [88]:
fromsyria<-syriaout %>% group_by(destination) %>% ungroup() %>% select("year","destination","total")

In [89]:
head(fromsyria)
dim(fromsyria)

year,destination,total
2010,United Arab Emirates,43
2010,Australia,86
2010,Austria,976
2010,Bulgaria,125
2010,Bosnia and Herzegovina,3
2010,Canada,766


[1] 662   3

In [19]:
data2 <- data %>% group_by(destination,origin) 

In [21]:
head(data2)

year,destination,origin,refuge,asylum,total
2010,Afghanistan,Iran (Islamic Rep. of),30,21,51
2010,Afghanistan,Pakistan,6398,9,6407
2010,Angola,Burundi,14,13,27
2010,Angola,Central African Rep.,6,1,7
2010,Angola,Côte d'Ivoire,211,1213,1424
2010,Angola,Dem. Rep. of the Congo,13648,1508,15156


In [37]:
data2 <- ungroup(data2)

destinations <- select(data2,destination)
origins <- select(data2,origin)

destinationdf <- distinct(as.data.frame(destinations))
origindf <- distinct(as.data.frame(origins))

In [56]:
head(origindf)
dim(origindf)
lapply(destinationdf,class)

origin
Iran (Islamic Rep. of)
Pakistan
Burundi
Central African Rep.
Côte d'Ivoire
Dem. Rep. of the Congo


[1] 195   1

$destination
[1] "character"

In [62]:
head(destinationdf)
dim(destinationdf)
class(destinationdf)
lapply(destinationdf,class)

destination
Afghanistan
Angola
Albania
United Arab Emirates
Argentina
Armenia


[1] 167   1

[1] "data.frame"

$destination
[1] "character"

In [63]:
locationsddf <- mutate_geocode(destinationdf, place)

ERROR: Error in geocode(locs, ...): is.character(location) is not TRUE


In [58]:
locationsodf <- mutate_geocode(origindf, place)

ERROR: Error in geocode(locs, ...): is.character(location) is not TRUE
